In [8]:
%pip install langchain transformers chromadb faiss-cpu pymupdf langchain-community

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
import faiss
import torch
from sentence_transformers import SentenceTransformer
from transformers import pipeline

In [10]:
file="final drug dataset.csv"
df = pd.read_csv(file)
df

,Interaction Description,Drug 1,Drug 2,Drug 3,Known Interaction
0,Trioxsalen may increase the photosensitizing a...,Trioxsalen,Verteporfin,None,Yes
1,Aminolevulinic acid may increase the photosens...,Aminolevulinic acid,Verteporfin,None,Yes
2,Titanium dioxide may increase the photosensiti...,Titanium dioxide,Verteporfin,None,Yes
3,Tiaprofenic acid may increase the photosensiti...,Tiaprofenic acid,Verteporfin,None,Yes
4,Cyamemazine may increase the photosensitizing ...,Cyamemazine,Verteporfin,None,Yes
...,...,...,...,...,...
221704,No side effects,Omeprazole,Clarithromycin,None,No
221705,No side effects,Vibegron,Tolterodine,None,No
221706,No side effects,Metformin,Sitagliptin,None,No
221707,No side effects,Albuterol,Ipratropium,None,No


In [11]:
required_columns = ["Drug 1", "Drug 2", "Drug 3", "Interaction Description"]
for col in required_columns:
    if col not in df.columns:
        raise ValueError(f"Missing column: {col}")

In [12]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")


In [13]:
df["combined_text"] = df["Drug 1"].fillna("") + " " + df["Drug 2"].fillna("") + " " + df["Drug 3"].fillna("") + " " + df["Interaction Description"].fillna("")

In [ ]:
embeddings = embedding_model.encode(df["combined_text"].tolist(), convert_to_numpy=True)

In [ ]:
# Load Embedding Model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
# Generate Embeddings
embeddings = embedding_model.encode(data["text"].tolist(), convert_to_numpy=True)


KeyboardInterrupt: 

In [ ]:
d = embeddings.shape[1]  # Dimensionality
index = faiss.IndexFlatL2(d)
index.add(embeddings)


In [ ]:
llm_pipeline = pipeline("text-generation", model="tiiuae/falcon-7b-instruct", torch_dtype=torch.float16, device=0)


In [ ]:
def find_safer_alternative(drug_query, condition_query):
    """Find safer drug alternatives based on similarity search and generate insights."""
    query_text = drug_query + " " + condition_query
    query_embedding = embedding_model.encode([query_text], convert_to_numpy=True)
    _, indices = index.search(query_embedding, k=3)  # Retrieve top 3 matches
    
    retrieved_drugs = df.iloc[indices[0]][["Drug 1", "Drug 2", "Drug 3"]].values.tolist()
    retrieved_texts = [" ".join(drugs) for drugs in retrieved_drugs]
    
    prompt = f"Given that {drug_query} is being used for {condition_query}, suggest safer alternatives based on similar cases: {retrieved_texts}."
    response = llm_pipeline(prompt, max_length=150)[0]["generated_text"]
    
    return response

In [ ]:
drug_query = "Ibuprofen"
condition_query = "Headache"
safer_alternative = find_safer_alternative(drug_query, condition_query)
print("Safer Alternative Suggestion:", safer_alternative)

NameError: name 'find_safer_alternative' is not defined